# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2024-2025, Q4
* Course homepage: https://brightspace.tudelft.nl/d2l/home/682445
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Niels Stienen (N.L.Stienen@student.tudelft.nl)
* (c) TU Delft, 2025

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE` and remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please **do not** delete it, instead run that cell as you would run all other cells. Finally, please **do not** add any extra cells to this notebook or change the existing cells unless you are explicitly asked to do so.

Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions (also after the course is finished), and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. Please note that it is **not** allowed to post any code relating to solution attempts. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA or if not possible otherwise, send an email to the instructor or TA.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for you that you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. 

* Please note that the above mentioned _read-only_ protection only works in Jupyter Notebook, and it does not work if you open this notebook in another editor (e.g., VSCode, PyCharm, etc.). Therefore, we recommend that you only use Jupyter Notebook for this course. If you use any other editor, you may accidentally delete cells, modify the tests, etc., which would cause you to lose points.

* If you edit a function that is imported in another notebook, you need to **restart the kernel** of the notebook where you are using the function. Otherwise, the changes will not be effective.

* **IMPORTANT**: Please make sure that your code executes without any errors before submitting the notebook. An easy way to ensure this is to use the validation script as described in the README.

# Lagrangian Neural Network (LNN)

**Author:** Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

This notebook will contain functions for implementing a Lagrangian neural network. Please follow the notebook `task_2c-2_lnn_implementation.ipynb`, which guides you through these implementations.

In [ ]:
from flax import linen as nn
from functools import partial
import jax

jax.config.update("jax_platforms", "cpu")  # set default device to 'cpu'
jax.config.update("jax_enable_x64", True)  # double precision
import jax
from jax import Array, jit, debug
import jax.numpy as jnp
import numpy as np
from typing import Dict, Tuple

from jax_double_pendulum.integrators import rk4_step
from jax_double_pendulum.utils import normalize_link_angles

In [ ]:
class MassMatrixNN(nn.Module):
    """
    Neural network to approximate the mass matrix.
    """

    num_hidden: int = 32  # Number of hidden units per intermediate layer

    diagonal_shift = 0.001  # shifting the diagonal before activation function
    diagonal_eps = 0.002  # small value added to the resulting diagonal

    @nn.compact
    def __call__(self, th: Array) -> Array:
        """
        Evaluate the mass matrix for the current neural network parameters
        Args:
            th: link angles of shape (2, ).
                We assume that the link angles are already normalized to the interval [-pi, pi].
        Returns:
            M: mass matrix of shape (2, 2)
        """
        num_dof = th.shape[-1]  # Degrees of Freedom of the robot
        num_nn_outputs = int((num_dof**2 + num_dof) / 2)

        # implement the neural network layers
        # use nn.Dense and nn.softplus layers
        # the output of the last layer needs to be saved in the variable `m`
        # the elements of the triangular matrix are the outputs of the network
        m = jnp.zeros((num_nn_outputs,))
        # YOUR CODE HERE
        raise NotImplementedError()

        # split-off the first num_dof as the diagonal entries
        l_diagonal, l_off_diagonal = jnp.split(
            m,
            np.array(
                [
                    num_dof,
                ]
            ),
            axis=-1,
        )

        # ensure positive diagonal
        # first, add `self.diagonal_shift` to the diagonal
        # then, apply softplus
        # finally, add `self.diagonal_eps` to the diagonal
        # YOUR CODE HERE
        raise NotImplementedError()

        # Calculate the indices of the diagonal elements of L:
        indices_diag = np.arange(num_dof, dtype=int) + 1
        indices_diag = (indices_diag * (indices_diag + 1) / 2 - 1).astype(int)  # [0, 2]
        # Calculate the indices of the off-diagonal elements of L:
        indices_off_diag = np.setdiff1d(np.arange(num_nn_outputs), indices_diag)  # [1]
        # Indexing for concatenation of l_diagonal and l_off_diagonal
        indices_nn_output = np.hstack((indices_diag, indices_off_diag))  # [0, 2, 1]

        # vector of lower triangular matrix (i.e. flattened lower triangular matrix)
        vec_tril = jnp.concatenate([l_diagonal, l_off_diagonal], axis=-1)[
            ..., indices_nn_output
        ]

        # construct empty triangular matrix
        tril_mat = jnp.zeros((num_dof, num_dof))
        # (i, j) indices of lower triangular matrix
        indices_tril = np.tril_indices(num_dof)  # (array([0, 1, 1]), array([0, 0, 1]))
        # populate triangular matrix from vector
        tril_mat = tril_mat.at[indices_tril].set(vec_tril[:])

        # construct mass matrix from triangular matrix
        M = tril_mat @ tril_mat.transpose()

        return M

In [ ]:
class PotentialEnergyNN(nn.Module):
    """
    Neural network to approximate the potential energy.
    """

    num_hidden: int = 32  # Number of hidden units per intermediate layer

    @nn.compact
    def __call__(self, th: Array) -> Array:
        """
        Evaluate the potential energy for the current neural network parameters
        Args:
            th: link angles of shape (2, ).
                We assume that the link angles are already normalized to the interval [-pi, pi].
        Returns:
            U: potential energy of shape ( )
        """
        # implement the neural network layers
        # Important: the output of the last layer needs to be a scalar (see docstring)
        # Hint: use jnp.squeeze to make sure the output is a scalar
        # YOUR CODE HERE
        raise NotImplementedError()

        return U

In [ ]:
@jit
def kinetic_energy_fn(
    mass_matrix_nn_params: Dict, th: Array, th_d: Array
) -> Array:
    """
    Compute the kinetic energy of the system using a learned neural-network-based mass matrix
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        th: link angles of shape (2, ).
            We assume that the link angles are already normalized to the interval [-pi, pi].
        th_d: link angular velocities of double pendulum of shape (2, )
    Returns:
        T: kinetic energy of shape ( )
    """
    # evaluate mass matrix at current system state
    M = MassMatrixNN().apply({"params": mass_matrix_nn_params}, th)

    # compute kinetic energy
    T = jnp.array(0)
    # YOUR CODE HERE
    raise NotImplementedError()
    print("T", T.shape)

    return T

In [ ]:
@jit
def potential_energy_fn(potential_energy_nn_params: Dict, th: Array) -> Array:
    """
    Compute the potential energy of the system using the `PotentialEnergyNN` neural network
    Args:
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        th: link angles of shape (2, ).
            We assume that the link angles are already normalized to the interval [-pi, pi].
    Returns:
        U: potential energy of shape ( )
    """
    # evaluate the `PotentialEnergyNN` at current system state
    U = jnp.array(0)
    # YOUR CODE HERE
    raise NotImplementedError()
    return U

In [ ]:
@jit
def lagrangian_fn(
    mass_matrix_nn_params: Dict,
    potential_energy_nn_params: Dict,
    th: Array,
    th_d: Array,
) -> Array:
    """
    Compute the Lagrangian of the system using a learned neural-network-based mass matrix and potential energy
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        th: link angles of shape (2, ).
            We assume that the link angles are already normalized to the interval [-pi, pi].
        th_d: link angular velocities of double pendulum of shape (2, )
    Returns:
        L: Lagrangian of shape ( )
    """
    # Compute the Lagrangian
    L = jnp.array(0)
    # Important: the Lagrangian needs to be a scalar (see docstring)
    # Hint: if necessary, use jnp.squeeze to make sure the output is a scalar
    # YOUR CODE HERE
    raise NotImplementedError()

    return L

In [ ]:
@jit
def mass_matrix_fn(mass_matrix_nn_params: Dict, th: Array) -> Array:
    """
    Compute the mass matrix of the system using the `MassMatrixNN` neural network
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        th: link angles of shape (2, ).
            We assume that the link angles are already normalized to the interval [-pi, pi].
    Returns:
        M: mass matrix of shape (2, 2)
    """
    M = jnp.zeros((2, 2))
    # YOUR CODE HERE
    raise NotImplementedError()
    return M


@jit
def dynamical_matrices(
    mass_matrix_nn_params: Dict,
    potential_energy_nn_params: Dict,
    th: Array,
    th_d: Array,
) -> Tuple[Array, Array, Array]:
    """
    Compute the dynamical matrices of the system using a learned neural-network-based
    mass matrix and potential energy
    The resulting Equations of Motion (EoM) are given by:
        M @ th_dd + C @ th_d + G = tau
    where tau are the external torques applied on the links.

    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        th: link angles of double pendulum of shape (2, )
        th_d: link angular velocities of double pendulum of shape (2, )

    Returns:
        M: mass matrix of shape (2, 2)
        C: coriolis and centrifugal matrix of shape (2, 2)
        G: gravity matrix of shape (2, )
    """
    # the neural network needs to be robust to the redundancy of th = th + n * 2*pi
    # therefore, we first project the given link angles onto the interval [-pi, pi]
    # Hint: use the `jax_double_pendulum.utils.normalize_link_angles` function.
    # Hint: use `jax.grad` or `jax.value_and_grad` for the 1st-order partial derivatives
    # Hint: use the `jax.hessian` function for computing the 2nd-order partial derivatives.
    # YOUR CODE HERE
    raise NotImplementedError()

    # Compute the dynamical matrices by taking the appropiate partial derivatives of the Lagrangian
    M, C, G = jnp.zeros((2, 2)), jnp.zeros((2, 2)), jnp.zeros((2,))
    # YOUR CODE HERE
    raise NotImplementedError()
    return M, C, G

In [ ]:
@jit
def continuous_forward_dynamics(
    mass_matrix_nn_params: Dict,
    potential_energy_nn_params: Dict,
    th: Array,
    th_d: Array,
    tau: Array = jnp.zeros((2,)),
) -> Array:
    """
    Compute the continuous forward dynamics of the system using a learned neural-network-based
    mass matrix and potential energy
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        th: link angles of double pendulum of shape (2, )
        th_d: link angular velocities of double pendulum of shape (2, )
        tau: link torques of double pendulum of shape (2, )
    Returns:
        th_dd: link angular accelerations of double pendulum of shape (2, )
    """
    # Compute the angular acceleration of the links
    th_dd = jnp.zeros((2,))
    # YOUR CODE HERE
    raise NotImplementedError()

    return th_dd

In [ ]:
def continuous_state_space_dynamics(
    mass_matrix_nn_params: Dict,
    potential_energy_nn_params: Dict,
    x: Array,
    tau: Array,
) -> Tuple[Array, Array]:
    """
    Compute the continuous forward dynamics of the system in state-space representation
    using the Lagrangian neural network
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        x: system state of shape (4, ) consisting of the link angles and velocities
        tau: link torques of shape (2, )
    Returns:
        dx_dt: time derivative of the system state of shape (4, )
        y: system output of shape (2, ) consisting of the link angles
    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return dx_dt, y

In [ ]:
@jit
def discrete_forward_dynamics(
    mass_matrix_nn_params: Dict,
    potential_energy_nn_params: Dict,
    dt: Array,
    th_curr: Array,
    th_d_curr: Array,
    tau: Array = jnp.zeros((2,)),
) -> Tuple[Array, Array, Array]:
    """
    Compute the discrete forward dynamics of the system using a learned neural-network-based
    mass matrix and potential energy
    Args:
        mass_matrix_nn_params: parameters of the MassMatrixNN
        potential_energy_nn_params: parameters of the PotentialEnergyNN
        dt: time step between the current and the next state [s] of shape ( )
        th_curr: current link angles of double pendulum of shape (2, )
        th_d_curr: current link angular velocities of double pendulum of shape (2, )
        tau: link torques of double pendulum of shape (2, )
    Returns:
        th_next: link angles at the next time step of double pendulum of shape (2, )
        th_d_next: link angular velocities at the next time step of double pendulum of shape (2, )
        th_dd: link angular accelerations between current and next time step of double pendulum of shape (2, )
    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return th_next, th_d_next, th_dd